Make world consist of cubes of equal size. This requires IDW to be "snapped" to the discritized 'z' axis.

In [21]:
import numpy as np
import matplotlib.pyplot as plt
from tools import parse_file, rescale, center, gen_borders
from interpol import Estimation

In [38]:
points = center(rescale(parse_file("../resources/demo2.mod1")))

In [39]:
points

array([[0.83333333, 0.83333333, 1.        ],
       [1.16666667, 0.83333333, 0.01666667],
       [1.16666667, 1.16666667, 0.66666667]])

In [46]:
xmin, xmax, ymin, ymax, zmin, zmax = 0, 2, 0, 2, 0, 2

In [47]:
points = np.vstack([points, gen_borders(0, 2, 0, 2)])

In [126]:
class Cube:
    def __init__(self, point, state="empty"):
        """ Cube is defined by a down-bottom-left point """
        self.point = point
        self.state = state
        
    def __repr__(self):
        return f"{self.point[2]:.2f} - {self.state}"
#         return f"({self.point[0]:.2f}, {self.point[1]:.2f}, {self.point[2]:.2f})"

In [176]:
NUM_POINTS = 40
x = np.linspace(0, 2, NUM_POINTS)
y = np.linspace(0, 2, NUM_POINTS)
z = np.linspace(0, 2, NUM_POINTS)
height_map = np.empty((NUM_POINTS, NUM_POINTS, NUM_POINTS), dtype=object)

In [177]:
z_step = zmax / NUM_POINTS

In [178]:
z_step

0.05

In [179]:
e = Estimation(points[:, 0], points[:, 1], points[:, 2])

In [190]:
height_map = np.empty((NUM_POINTS, NUM_POINTS, NUM_POINTS), dtype=object)
for i in range(NUM_POINTS):
    for j in range(NUM_POINTS):
        z_interpol = e.estimate(x[i], y[j])  # real-valued
        z_discrete = int(z_interpol // z_step)
        for k, _ in enumerate(z):
            if k == z_discrete:
                height_map[i, j, k] = Cube([x[i], y[j], _], 'terrain')
            else:
                height_map[i, j, k] = Cube([x[i], y[j], _])
#         print(z_interpol, z_interpol // z_step)

In [192]:
height_map[10]

array([[0.00 - terrain, 0.05 - empty, 0.10 - empty, ..., 1.90 - empty,
        1.95 - empty, 2.00 - empty],
       [0.00 - terrain, 0.05 - empty, 0.10 - empty, ..., 1.90 - empty,
        1.95 - empty, 2.00 - empty],
       [0.00 - terrain, 0.05 - empty, 0.10 - empty, ..., 1.90 - empty,
        1.95 - empty, 2.00 - empty],
       ...,
       [0.00 - terrain, 0.05 - empty, 0.10 - empty, ..., 1.90 - empty,
        1.95 - empty, 2.00 - empty],
       [0.00 - terrain, 0.05 - empty, 0.10 - empty, ..., 1.90 - empty,
        1.95 - empty, 2.00 - empty],
       [0.00 - terrain, 0.05 - empty, 0.10 - empty, ..., 1.90 - empty,
        1.95 - empty, 2.00 - empty]], dtype=object)

In [ ]:
def idw_mesh(points, n_points=50, sigma=0.6, map_points=True, xmin=0, xmax=2, ymin=0, ymax=2):
    """
        Generate mesh of interpolated height map using Inverse Distance Weighting

    :param points: map points to interpolate
    :param n_points: number of points per axis. 50 will give you 50x50 grid
    :param sigma: smoothing parameter
            Euclidean distance d(p1, p2) ^ (1 + sigma)
    :param map_points: Append map points if True
    :param xmin:
    :param xmax:
    :param ymin:
    :param ymax:
    :return: Array of interpolated points
    """
    e = Estimation(points[:, 0], points[:, 1], points[:, 2])
    x = np.linspace(xmin, xmax, n_points)
    y = np.linspace(ymin, ymax, n_points)
    res = np.zeros((n_points ** 2, 3))
    for i in range(n_points):
        for j in range(n_points):
            res[i * n_points + j, :] = x[i], y[j], e.estimate(x[i], y[j], sigma=sigma)
    if map_points:
        return np.vstack((res, points))
    return res